<a href="https://colab.research.google.com/github/EmilSeyfullayev/Mercedes/blob/main/2_DeactivatedCarsParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #About first threee cells
# #In order to use colab with google drive with no authentication each time you have run this cell
# #Then you will have to run next cell to copy client_id, client secret and refresh_token
# #once you did thah you paste them in third cell and by running third cell only you dont need to authenticate each time
# # while third cell is run, adc.json file is created
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !cat adc.json

{
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//04H-Te-Q7hCNgCgYIARAAGAQSNwF-L9IrDIZbW-ptI2cm7quh8_vQvBYuzqWw8V98Bvp5_dz5-F0wR2LH5-ZDP_D7T8b8eK6DlU0",
  "revoke_uri": "https://accounts.google.com/o/oauth2/revoke",
  "scopes": [
    "openid",
    "https://www.googleapis.com/auth/userinfo.email",
    "https://www.googleapis.com/auth/cloud-platform",
    "https://www.googleapis.com/auth/appengine.admin",
    "https://www.googleapis.com/auth/compute",
    "https://www.googleapis.com/auth/accounts.reauth",
    "https://www.googleapis.com/auth/drive"
  ],
  "token_uri": "https://oauth2.googleapis.com/token",
  "type": "authorized_user"
}

In [ ]:
start_number = -3000

################################################
!pip install -U -q PyDrive

import httplib2
import json

from google.colab import auth
from oauth2client import GOOGLE_REVOKE_URI, GOOGLE_TOKEN_URI, client
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

auth_key = {
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//04H-Te-Q7hCNgCgYIARAAGAQSNwF-L9IrDIZbW-ptI2cm7quh8_vQvBYuzqWw8V98Bvp5_dz5-F0wR2LH5-ZDP_D7T8b8eK6DlU0"
  }

credentials = client.OAuth2Credentials(
    access_token=None,
    client_id=auth_key['client_id'],
    client_secret=auth_key['client_secret'],
    refresh_token=auth_key['refresh_token'],
    token_expiry=None,
    token_uri=GOOGLE_TOKEN_URI,
    user_agent=None,
    revoke_uri=GOOGLE_REVOKE_URI)

credentials.refresh(httplib2.Http())
credentials.authorize(httplib2.Http())
cred = json.loads(credentials.to_json())
cred['type'] = 'authorized_user'

with open('adc.json', 'w') as outfile:
  json.dump(cred, outfile)

auth.authenticate_user()

###############################################################################
from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd
import numpy as np
from datetime import timezone, tzinfo
import pytz
import datetime         # datetime.datetime.now()
import time             # time.sleep(int(secs))

HOST = "https://turbo.az/"
HEADERS = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

#################################################################################
import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())

db_ws = gc.open("MercedesDataBase").worksheet("sheet_1")
db_data = gd.get_as_dataframe(db_ws)

list_of_links = list(db_data['Link'])

print("Number of cars existing in db: " + str(len(list_of_links))) 

###############################################################################

#If car in link is deactivated it will not be used anymore
ws_deactivated = gc.open("DeactivatedCars").worksheet("sheet_1")
db_deactivated_data = gd.get_as_dataframe(ws_deactivated)

list_of_deactivated_links = list(db_deactivated_data['Link'])

list_of_links_to_check_deactivation = []

for link in list_of_links:
  if link not in list_of_deactivated_links:
    list_of_links_to_check_deactivation.append(link)

print("Number of links to check deactivation: " + str(len(list_of_links_to_check_deactivation))) 

###############################################################################

descriptions = []
dates = []
number_of_looks = []
productS_id = []
phone_book = []
avtosalon_name = []
extras = []
product_text = []
location = []
time_parsed = []
date_parsed = []

def car_properties_parser():

  for link in list_of_links_to_check_deactivation[-42000:-41500]:

    requested_html = requests.get(link, headers=HEADERS).content
    parsed_page = BeautifulSoup(requested_html, "html.parser")

    try:

      #this is for date inside of the elan
      dates.append(parsed_page.find("div", class_="product-statistics").find_all("p")[1].text.split(": ")[1])

      #this is for parsing number of looks
      looks_quantity = parsed_page.find_all("div", {"class":"product-statistics"})[0].find_all("p")[0].text.split(" ")[2]
      number_of_looks.append(looks_quantity)



      #this is for 14 important descriptions of a car
      for main_description_division in parsed_page.find_all("div", class_="product-properties-value")[0:14]: #14 in order not to consider Barter and kredit
        descriptions.append(main_description_division.text)
      

      #this is for parsing products id
      product_id = parsed_page.find_all("div", {"class":"product-statistics"})[0].find_all("p")[2].text.split(" ")[2]
      productS_id.append(product_id)

      #for parsing product text
      try:
        product_text.append(parsed_page.find(class_="product-text").text)
      except AttributeError:
        product_text.append("No text for product")

      #for parsing extras
      try:
        extras.append(parsed_page.find("div", class_="product-extras").text)
      except AttributeError:
        extras.append("No extras for product")

      #this is for parsing phone numbers

      try:
        phone_book.append(parsed_page.find(class_='phone').text)

      except AttributeError:
        phones = []
        for i in parsed_page.find_all(class_='shop-contact--phones-i'):
          phones.append(i.text)
        
        phone_book.append(phones)

      
      #for avtosalon name
      
      try:
        avtosalon_name.append(parsed_page.find(class_="shop--title").text)
      except AttributeError:
        avtosalon_name.append(0)

      try:
        location.append(parsed_page.find(class_="shop-contact--location-map shop--open-location-modal").find("img").get('src').split("markers=")[1].split("&size")[0])
      except AttributeError:
        location.append(0)
      


      date_parsed.append(datetime.datetime.now(pytz.timezone("Asia/Baku")).strftime("%H:%M:%S"))
      time_parsed.append(datetime.datetime.now(pytz.timezone("Asia/Baku")).strftime("%Y-%m-%d"))

      print("Parsed link number is " + str(list_of_links_to_check_deactivation.index(link)+1)+ " out of "+ str(len(list_of_links_to_check_deactivation))+  " links.")
      time.sleep(1)

    except AttributeError:

      descriptions.extend(list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
      dates.append("duplicate_or_no_car")
      number_of_looks.append("duplicate_or_no_car")
      productS_id.append("duplicate_or_no_car")
      phone_book.append("duplicate_or_no_car")
      avtosalon_name.append("duplicate_or_no_car")
      extras.append("duplicate_or_no_car")
      product_text.append("dupduplicate_or_no_carlicate")
      location.append("duplicate_or_no_car")
      time_parsed.append("duplicate_or_no_car")
      date_parsed.append("duplicate_or_no_car")

      print("Duplicates or no cars were parsed in link "+ str(link))
    
    # except AttributeError:

    #   descriptions.extend(list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    #   dates.append("No car")
    #   number_of_looks.append("No car")
    #   productS_id.append("No car")
    #   phone_book.append("No car")
    #   avtosalon_name.append("No car")
    #   extras.append("No car")
    #   product_text.append("No car")
    #   location.append("No car")
    #   time_parsed.append("No car")
    #   date_parsed.append("No car")

    #   print("No car was parsed in link "+ str(link))


car_properties_parser()


Number of cars existing in db: 51814
Number of links to check deactivation: 45210
Parsed link number is 3211 out of 45210 links.
Parsed link number is 3212 out of 45210 links.
Parsed link number is 3213 out of 45210 links.
Parsed link number is 3214 out of 45210 links.
Parsed link number is 3215 out of 45210 links.
Parsed link number is 3216 out of 45210 links.
Parsed link number is 3217 out of 45210 links.
Parsed link number is 3218 out of 45210 links.
Parsed link number is 3219 out of 45210 links.
Parsed link number is 3220 out of 45210 links.
Parsed link number is 3221 out of 45210 links.
Parsed link number is 3222 out of 45210 links.
Parsed link number is 3223 out of 45210 links.
Parsed link number is 3224 out of 45210 links.
Parsed link number is 3225 out of 45210 links.
Parsed link number is 3226 out of 45210 links.
Parsed link number is 3227 out of 45210 links.
Parsed link number is 3228 out of 45210 links.
Parsed link number is 3229 out of 45210 links.
Parsed link number is 323

In [ ]:
# descriptions = a
# dates = b
# number_of_looks = c
# productS_id = d
# phone_book = e
# avtosalon_name = f
# extras = g
# product_text = h
# location = i
# time_parsed = j
# date_parsed = k

In [ ]:
## Should be same length
print(int(len(descriptions)/14))
print(len(number_of_looks))
print(len(productS_id))
print(len(phone_book))
print(len(avtosalon_name))
print(len(extras))
print(len(product_text))
print(len(location))
print(len(time_parsed))
print(len(date_parsed))
print(len(list_of_links_to_check_deactivation[start_number:]))
print(len(dates))

####Naming columns


first_column_names = []

html = requests.get(list_of_links[0], headers=HEADERS).content
soup = BeautifulSoup(html, "html.parser")

for labels in soup.find_all("li", class_="product-properties-i"):
  for label in labels.find_all("label"):
    first_column_names.append(label.text)

data = pd.DataFrame(np.array(descriptions).reshape(-1, 14))

data.columns = first_column_names

data["Tarix"] = dates

data['Baxışların sayı'] = number_of_looks
data['Məhsulun id-si'] = productS_id
data['Link'] = list_of_links_to_check_deactivation[-42000:-41500]

data['Əlaqə nömrəsi'] = phone_book
data['Avtosalon adı'] = avtosalon_name
data['Ekstra parametrlər'] = extras
data['Məhsulun mətni'] = product_text
data['Yerləşmə məkanı'] = location
data['Parsinq olunan tarix'] = date_parsed
data['Parsinq olunan zaman'] = time_parsed 

############################################################################
## do not reverse order of cars cause they are already reversed
checkpoint = data
##  data = data.iloc[::-1].reset_index(drop=True)

############################################################################

deactivated_index = []

for i in list(data['Əlaqə nömrəsi']):
  if i == []:
    deactivated_index.append(1)
  elif i != []:
    deactivated_index.append(0)

############################################################################
##separation of parsed data
data['deactivated_index'] = deactivated_index
data_deactivated = data[data['deactivated_index']==1]
duplicate_or_no_car = data[data['Tarix'] == "duplicate_or_no_car"]

print("Number of cars deactivated: " + str(len(data_deactivated)))

############################################################################
##this is to write information for the first time
import gspread_dataframe as gd
import gspread as gs

gc = gs.authorize(GoogleCredentials.get_application_default())
# ws = gc.open("DeactivatedCars").worksheet("sheet_1")
# gd.set_with_dataframe(ws, data_deactivated, resize=True)

#This is to add new data_existed_in_db_necessary_columns to existing database

ws_deactivated_db = gc.open("DeactivatedCars").worksheet("sheet_1")
print("Length of deactivated cars to be added:" + str(data_deactivated.shape[0]))
ws_deactivated_db.add_rows(data_deactivated.shape[0])
gd.set_with_dataframe(worksheet=ws_deactivated_db, dataframe=data_deactivated, 
                       include_index=False, include_column_header=False,
                       row=ws_deactivated_db.row_count+1)
##############################################################################

## Duplicate or no car
##this is to write information for the first time
import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())
# ws = gc.open("DeactivatedCars").worksheet("sheet_1")
# gd.set_with_dataframe(ws, duplicate_or_no_car, resize=True)

#This is to add new data_existed_in_db_necessary_columns to existing database

ws_deactivated_db = gc.open("DeactivatedCars").worksheet("sheet_1")
print("Length of duplicates or no cars to be added:" + str(duplicate_or_no_car.shape[0]))
ws_deactivated_db.add_rows(duplicate_or_no_car.shape[0])
gd.set_with_dataframe(worksheet=ws_deactivated_db, dataframe=duplicate_or_no_car, 
                       include_index=False, include_column_header=False,
                       row=ws_deactivated_db.row_count+1)

################################################################################
second_updated_db = data[data['Tarix'] != "duplicate_or_no_car"]

gc = gs.authorize(GoogleCredentials.get_application_default())
# ws_second_updated = gc.open("Second_updated").worksheet("sheet_1")
# gd.set_with_dataframe(ws_second_updated, second_updated_db, resize=True)

#This is to add new data_existed_in_db_necessary_columns to existing database

ws_second_updated = gc.open("Second_updated").worksheet("sheet_1")
print("Length of data to be added to second updated db: " + str(second_updated_db.shape[0]))
ws_second_updated.add_rows(second_updated_db.shape[0])
gd.set_with_dataframe(worksheet=ws_second_updated, dataframe=second_updated_db, 
                        include_index=False, include_column_header=False,
                        row=ws_second_updated.row_count+1)

500
500
500
500
500
500
500
500
500
500
3000
500
Number of cars deactivated: 489
Length of deactivated cars to be added:489
Length of duplicates or no cars to be added:5
Length of data to be added to second updated db: 495


In [ ]:
# ##Prints sum of not existed cars vs existing in db
# data_deactivated = data[data['Əlaqə nömrəsi']==a]
# print(len(data_deactivated))
# data_active = data[data['Əlaqə nömrəsi']!=a]
# print(len(data_active))